# Supplementary figure about the IBCM model on toy backgrounds
Convergence time, eigenvalues and saddle point in the first phase of the dynamics, necessity of a third moment in the background statistics to break degeneracy of fixed points (non-isolated fixed points). 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder1 = "panels1/"
panels_folder2 = "panels2/"
panels_folder3 = "panels3/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

# Panels A-D: IBCM dynamics in 2D toy model
This is a figure previously made to illustrate IBCM. 
A: illustrate background
B: M weights and analytics
C: W weights and analytics
D: habituation performance

In [ ]:
_, n_neu, n_orn = np.load(pj(data_folder, "sample_2d_simulation.npz"))["mbarser"].shape

In [ ]:
# Extra aesthetic parameters for this figure
# More legend rcParams: make everything smaller by 30 %
#plt.rcParams["patch.linewidth"] = 0.75
#legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
#             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
#for k in legend_rc:
#    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

neuron_colors = neuron_colors_full24[[8, 17]]
new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

## Panel A: OSN time series

In [ ]:
# Load example concentration time series
ex = np.load(os.path.join(data_folder, "sample_2d_simulation.npz"))
skp_example = 1
tser_example = ex["tser"]  # Unskip this
save_skp = ex["save_skp"][0]
tser_example_unskp = np.arange(tser_example[0], tser_example[-1]+np.mean(np.diff(tser_example)), 
                               (tser_example[1]-tser_example[0])/save_skp)
                               
nuser_example = ex["nuser"]  # This has all time steps, for nicer histogram
lim = 1e-5
nuser_example_clipped = np.clip(nuser_example, a_min=lim-0.5, a_max=0.5-lim)
#ratio_ser = ((0.5 - nuser_example_clipped) / (0.5 + nuser_example_clipped))
ratio_ser = ((0.5 - nuser_example) / (0.5 + nuser_example)).flatten()

# Time units for plotting, in ms
dt_u = 10.0  # ms

In [ ]:
# Plot time series and histogram
fig = plt.figure()
gs = fig.add_gridspec(1, 3)
axes = [fig.add_subplot(gs[:2])]
axes.append(fig.add_subplot(gs[2], sharey=axes[0]))
# Plot time series
t_window = slice(25000, 26500 // skp_example, 1)
tser_clipped = tser_example_unskp[ratio_ser > 0.0]
tser_clipped = tser_clipped[t_window]
ratio_ser_clipped = ratio_ser[ratio_ser > 0.0]
ax = axes[0]
ax.plot((tser_clipped - tser_clipped[0])*dt_u*skp_example/1000, 
        ratio_ser_clipped[t_window], color=back_color_samples, lw=0.5)
ax.set_xlabel("Time (s)")
ax.set_ylabel(r"Ratio of odors $\left(\frac{1 - 2\nu(t)}{1 + 2\nu(t)} \right)$", labelpad=0.0)
ax.set_yscale("log")
ax.set_ylim([1e-3, 1e3])
ax.set_xticks([0, 5, 10, 15])

# Plot a small histogram
counts, bins = np.histogram(np.log10(ratio_ser_clipped), bins="sqrt", density=True)
bins = 10.0**bins
ax = axes[1]
ax.barh(bins[:-1], counts, align="edge", height=bins[1:] - bins[:-1], color=back_color_samples)
ax.set(xscale="log", yscale="log", xlabel="Prob. density", xlim=[5e-4, 2e0])
ax.tick_params(axis="x", which="major", pad=1.0)
[label.set_visible(False) for label in ax.get_yticklabels()]
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder1, "supfig_ibcm1_sample_toy_odor_ratio_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Alternate panel A: 2D background manifold

In [ ]:
back_vecs_example = ex["back_vecs"]
s_nuser_example = ex["nuser"]  # This has all time steps, for nicer histogram
sd_ss_vecs = np.asarray([
    0.5 * np.sum(back_vecs_example, axis=0), 
    np.diff(back_vecs_example, axis=0).flatten()
])
back_ser = sd_ss_vecs[0] + sd_ss_vecs[1] * s_nuser_example
back_ser.shape

In [ ]:
skp_plot = 200

# 3D plot of the synaptic weight vectors of the different neurons
fig, ax = plt.subplots()
ax.plot(0, 0, color="k", marker="o", ls="none", ms=2)
tslice = slice(0, back_ser.shape[0], skp_plot)
scale = 100
ax.plot(back_ser[tslice, 0], back_ser[tslice, 1], marker="o", ms=4.0, mec="k", mew=0.2,
        alpha=0.6, color=back_color_samples, lw=2.0, ls="none")
#ax.tick_params(pad=-3)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel(r"${s}_1$", labelpad=4)
ax.set_ylabel(r"${s}_2$", labelpad=4)

# Annotate vectors
aprops = dict(arrowstyle="<-", color="k", lw=1)
input_vecs = back_vecs_example
lbls = [r"$\mathbf{s}_a$", r"$\mathbf{s}_b$", r"$\mathbf{s}_d$"]
for lbl, vec in zip(lbls, [*input_vecs[:, :2]] + [sd_ss_vecs[0, :2]]):
    ax.annotate("", xy=(0, 0), xytext=vec, arrowprops=aprops, xycoords="data", ha="center")
    ax.annotate(lbl, xy=(0, 0), xytext=vec*0.6*np.array([0.9, 1.0]), xycoords="data", ha="right")
ax.annotate("", xy=sd_ss_vecs[0, :2], xytext=sd_ss_vecs[1, :2]+sd_ss_vecs[0, :2], 
            arrowprops=aprops, xycoords="data", ha="center")
ax.annotate(r"$\mathbf{s}_s$", xy=sd_ss_vecs[0, :2], 
            xytext=(sd_ss_vecs[1, :2]+sd_ss_vecs[0, :2])*np.array([0.95, 1.35]), 
            xycoords="data", ha="right", va="bottom")

axlabels = (ax.xaxis.label, ax.yaxis.label)
if do_save_plots:
    fig.savefig(pj(panels_folder1, "supfig_ibcm1_toy_background_vectors.pdf"), 
               transparent=True,  bbox_inches="tight", bbox_extra_artists=axlabels)
plt.show()
plt.close()

## Panel B: $M$ series

In [ ]:
back_vecs_example = ex["back_vecs"]
n_b = back_vecs_example.shape[0]
mbarser_example = ex["mbarser"]
cbarser_example = ex["cbarser"]
sser_example = ex["sser"]
wser_example = ex["wser"]
cgammaser_example = mbarser_example.dot(back_vecs_example.T)

# Analytical predictions
analytical_barm = ex["analytical_barm"] 
analytical_cgammas = analytical_barm.dot(back_vecs_example.T)
analytical_w = ex["analytical_w"].T
print(analytical_cgammas)

In [ ]:
tser_example.shape

In [ ]:
skp_plot = 10
tslice = slice(0, tser_example.shape[0], skp_plot)

fig, ax = plt.subplots()
ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
t_axis = tser_example*dt_u/1000/60
legend_styles = [[0, 0], [0, 0]]
for i in range(n_neu):
    clr = neuron_colors[i]
    for j in range(n_b):
        #clr = neuron_colors[i]
        li, = ax.plot(t_axis[tslice], cgammaser_example[tslice, i, j], color=clr, 
                ls="-", alpha=1.0-0.5*j, lw=plt.rcParams["lines.linewidth"]-j*0.4)
        legend_styles[i][j] = li
cgammas_names = [r"Analytical $\bar{h}_\mathrm{ns}$", r"Analytical $\bar{h}_\mathrm{sp}$"]
for j in range(2):
    ax.axhline(analytical_cgammas[0][j], lw=1.0, ls="-.", color="grey")
    ax.annotate(cgammas_names[j], xy=(t_axis[5], analytical_cgammas[0][j]), 
                ha="left", va="bottom", color="grey", size=6)
ax.axhline(1.0, xmax=0.4, lw=1.0, ls=":", color="grey")
ax.annotate("Saddle", xy=(t_axis[5], 1.1), ha="left", va="bottom", color="grey", size=6)
ax.set(xlabel="Time (min)", 
       ylabel=r"Alignment $\bar{h}_{i \gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{x}_{\gamma}$")
ax.set_xticks(np.arange(0.0, t_axis[-1], 2.0))

# Need to do tight_layout before adding inset
fig.tight_layout()

# Custom legend: 2x2 table here, inset axes
axi = inset_axes(ax, width="50%", height="35%",
                   bbox_to_anchor=(.65, .32, .6, .5),
                   bbox_transform=ax.transAxes, loc=3)
axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for i in range(n_neu):
    for j in range(n_b):
        li = legend_styles[i][j]
        axi.plot([0.0+j, 0.5+j], [i, i], color=li.get_color(), 
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
for side in ["bottom", "left", "top", "right"]:
    axi.spines[side].set_visible(False)
axi.tick_params(axis="both", length=0, pad=5)
axi.set_xticks([0.25, 1.25])
#axi.set_xticklabels([r"$\mathbf{\bar{m}}^i \cdot \mathbf{x}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{x}_b$"])
axi.set_xticklabels([r"$\gamma=a$", r"$\gamma=b$"])
axi.set_xlabel("Component", size=6, labelpad=3)
axi.xaxis.set_label_position('top') 
axi.set_yticks([0, 1])
axi.invert_yaxis()
axi.set_yticklabels(["$i=1$", "$i=2$"])
#axi.set_yticklabels(["Neuron 2", "Neuron 1"])
axi.set_ylabel("Neuron", size=6, labelpad=3)
if do_save_plots:
    fig.savefig(pj(panels_folder1, "supfig_ibcm1_sample_toy_odor_m_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel C: $W$ series

In [ ]:
skp_lbl = n_orn // 4
lbl = "i={}".format(i) if i % skp_lbl == 0 else ""
skp_plot = 20
tslice = slice(0, tser_example.shape[0], skp_plot)
t_axis = tser_example*dt_u/1000/60

fig, axes = plt.subplots(1, n_neu, sharex=True, sharey=True)
axes = axes.flatten()
# Make a palette for each neuron
neuron_comp_palettes = [sns.light_palette(a, n_colors=n_orn) for a in neuron_colors]
analytic_colors = [a[len(a)//2] for a in neuron_comp_palettes]
for i in range(n_orn):
    for j in range(n_neu):
        ax = axes[j]
        li, = ax.plot(t_axis[tslice], wser_example[tslice, i, j], zorder=analytical_w.size + i*n_orn + j, 
                      color=neuron_comp_palettes[j][i])
        lbl = "Analytical" if i == 0 else ""
        ax.axhline(analytical_w[i, j], color=analytic_colors[j], ls="--", lw=0.65, label=lbl)
for j in range(n_neu):
    axes[j].set_title("Neuron $j = {}$".format(j+1))
    axes[j].set_xlabel("Time (min)")
axes[1].set_ylim(axes[1].get_ylim()[0], axes[1].get_ylim()[1]*1.15)
axes[0].legend(frameon=False)
axes[1].legend(frameon=False)
axes[0].set_ylabel(r"Inhibitory weights $W_{ij}$")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder1, "supfig_ibcm1_sample_toy_odor_w_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Panel D: background inhibition

In [ ]:
bkvecser_example = (0.5+nuser_example)*ex["back_vecs"][0] + (0.5 - nuser_example)*ex["back_vecs"][1]
bkvecser_example = bkvecser_example[::int(ex["save_skp"][0])]
norm_s2 = np.sum(sser_example**2, axis=1)
norm_x2 = np.sum(bkvecser_example**2, axis=1)
norm_s_analytic = ex["analytical_factor"][0] * np.sqrt(np.mean(norm_x2))

skp_plot = 4
tslice = slice(0, tser_example.shape[0], skp_plot)

fig, ax = plt.subplots()
ax.plot(t_axis[tslice], np.sqrt(norm_x2[tslice]), color=back_color, label=r"Background $\|\mathbf{s}\|$", lw=0.5)
ax.plot(t_axis[tslice], np.sqrt(norm_s2[tslice]), color=model_colors["ibcm"], label=r"IBCM $\|\mathbf{y}\|$", lw=0.5)
ax.set(xlabel="Time (min)", ylabel=r"PN activity norm, $\|\mathbf{y}\|$")
ax.set_xticks(np.arange(0.0, t_axis[-1], 2.0))
ax.axhline(norm_s_analytic, ls="-.", label=r"Analytical $\langle \|\mathbf{y}\| \rangle$",
           color=(mpl.colors.to_rgba(model_colors["ibcm"]) + np.asarray([0.25,]*3 + [0])).clip(0, 1))
ax.legend(bbox_to_anchor=(1.0, 0.2), loc="lower right", frameon=False)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder1, "supfig_ibcm1_sample_toy_odor_y_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panels E-H: convergence time in 2D toy model
4 sub-panels for convergence time analysis in the 2D toy model

In [ ]:
convtime_res = np.load(pj(data_folder, "convergence_time_2d_simulation.npz"))

In [ ]:
# Plot the example
def plot_convergence_dynamics(tser_t, hsers, abpreds, hdspreds, tdspreds_t):
    dt_u = 10.0  # time step = 10 ms
    time_fact =  dt_u / 1000 / 60  # min
    tser = tser_t * time_fact
    tdspreds = tdspreds_t * time_fact
    fig, axes = plt.subplots(1, 2)
    fig.set_size_inches(2*plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])
    axes = axes.flatten()
    ax = axes[0]
    # Plot dot products with vectors x_a and x_b
    skpt = 10  # Saved series have already skipped steps
    li1, = ax.plot(tser[::skpt], hsers[0][::skpt], color=neuron_colors[1],
            label=r"$h_{\mathrm{a}} = \mathbf{m}(t) \cdot \mathbf{s}_\mathrm{a}$")
    ax.plot(tser[::skpt], hsers[1][::skpt], color=neuron_colors[1], alpha=0.6, lw=li1.get_linewidth()*0.8,
            label=r"$h_{\mathrm{b}} = \mathbf{m}(t) \cdot \mathbf{s}_\mathrm{b}$")
    
    # Analytical predictions
    ax.axhline(abpreds[0], ls=":", color="k")
    ax.annotate(r"$h_{\mathrm{a}} = 1 + 1/(2 \sigma)$", xy=(tser[-1], abpreds[0]*1.1), 
                ha="right", va="bottom", fontsize=plt.rcParams["legend.fontsize"])
    ax.axhline(abpreds[1], ls=":", color="k")
    ax.annotate(r"$h_{\mathrm{b}} = 1 - 1/(2 \sigma)$", xy=(tser[-1], abpreds[1]*1.4), 
                ha="right", va="top", fontsize=plt.rcParams["legend.fontsize"])
    ax.set(xlabel="Time", ylabel=r"Alignment $h_{\mathrm{a, b}}$")
    ax.set_ylim(ax.get_ylim()[0]*1.2, ax.get_ylim()[1])
    
    # Plot dot products with x_s and x_d
    ax = axes[1]
    ax.plot(tser[::skpt], hsers[3][::skpt], color="xkcd:light orange", alpha=0.6,
            label=r"$h_{\mathrm{s}} = \mathbf{m}(t) \cdot \mathbf{s}_\mathrm{s}$")  # (\mathbf{s}_\mathrm{a} + \mathbf{s}_\mathrm{b})/2
    ax.plot(tser[::skpt], hsers[2][::skpt], color="grey", alpha=0.6, 
            label=r"$h_{\mathrm{d}} = \mathbf{m}(t) \cdot \mathbf{s}_\mathrm{d}$")  # (\mathbf{s}_\mathrm{a} - \mathbf{s}_\mathrm{b})
    
    hline_color = "xkcd:burgundy"
    ax.axhline(hdspreds[1], ls=":", color=hline_color)  # r"$\mathbf{m} \cdot \mathbf{s}_\mathrm{s} = \pm 1/\sigma$"
    ax.annotate(r"$h_\mathrm{s} = \pm 1/\sigma$", xy=(tser[-1], hdspreds[1]*1.05), color=hline_color,
                ha="right", va="bottom", fontsize=plt.rcParams["legend.fontsize"])
    hline_color = "k"
    ax.axhline(hdspreds[0], color=hline_color, ls=":")  # r"$\mathbf{m} \cdot \mathbf{s}_\mathrm{d} = 1$"
    ax.annotate(r"$h_\mathrm{d} = 1$", xy=(tser[-1], hdspreds[0]*0.7), color=hline_color,
                ha="right", va="top", fontsize=plt.rcParams["legend.fontsize"])
    ax.set(xlabel="Time (min)", ylabel=r"Alignment $h_{\mathrm{d, s}}$")
    # Convergence times predicted
    for ax in axes:
        ax.axvline(tdspreds[1], color="xkcd:dark orange", ls="--", 
                   label=r"Pred. $t_\mathrm{s}$ (selective)", lw=li1.get_linewidth()*1.25)
        ax.axvline(tdspreds[0], color="xkcd:dark grey", ls="-.", 
                   label=r"Pred. $t_\mathrm{d}$ (saddle)", lw=li1.get_linewidth()*1.25)

    axes[0].legend(loc='center right', handlelength=2.0, frameon=False)
    axes[1].legend(loc='center right', bbox_to_anchor=(1.0, 0.6), handlelength=2.0, frameon=False)
    return fig, axes

In [ ]:
def plot_convergence_time_scaling(eps_axes, time_grids):
    dt_u = 10.0  # time step = 10 ms
    time_fact =  dt_u / 1000 / 60  # min
    n_tries = eps_axes[0].size
    epsd_axis, epss_axis = eps_axes
    td_grid_t, ts_grid_t = time_grids
    td_grid = time_fact * td_grid_t
    ts_grid = time_fact * ts_grid_t
    leg_kwargs = dict(title="Simulation", frameon=False, 
        title_fontsize=plt.rcParams["legend.fontsize"], 
        fontsize=plt.rcParams["legend.fontsize"],
        bbox_to_anchor=(1.0, 1.07), loc="upper right", 
        labelspacing=0.02
    )
    
    # Plot results
    fig, axes = plt.subplots(1, 2)
    fig.set_size_inches(2*plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])
    axes = axes.flatten()
    # td as a function of epsd; should not depend on eps_s
    ax = axes[0]
    colors = sns.color_palette("mako", n_colors=n_tries)
    for j in range(epsd_axis.size):
        ax.plot(epsd_axis, td_grid[1, :, j], color=colors[j], ls="--",
                label=r"$\epsilon_\mathrm{s}" + "= {:.3f}$".format(epss_axis[j]), marker="o", ms=4)
    # Plot theory, independent of eps_s
    ax.plot(epsd_axis, td_grid[0, :, 0], label="Analytical", color="k", ls="-", lw=1.5)
    ax.set(xlabel=r"$\epsilon_\mathrm{d}$ (initial $h_\mathrm{d} = \mathbf{m} \cdot (\mathbf{s}_\mathrm{a} + \mathbf{s}_\mathrm{b})/2$)", 
           ylabel=r"$t_\mathrm{d}$ (min)",   #$\mathbf{m} \cdot \mathbf{x}_\mathrm{d}$ convergence time)
           title="First phase: to saddle point", xscale="log", yscale="log")
    ax.legend(**leg_kwargs)
    
    # ts - td as a function of epss
    ax = axes[1]
    colors = sns.color_palette("flare", n_colors=n_tries)
    # Plot simulations
    for i in range(epsd_axis.size):
        ax.plot(epss_axis, ts_grid[1, i] - td_grid[1, i], color=colors[i], ls="--",
                label=r"$\epsilon_\mathrm{d}" + "= {:.3f}$".format(epsd_axis[i]), marker="o", ms=4)
    # Plot theory, independent of eps_s
    ax.plot(epss_axis, ts_grid[0, 0] - td_grid[0, 0], label="Analytical", color="k", ls="-", lw=1.5)
    ax.set(xlabel=r"$\epsilon_\mathrm{s}$ (initial $h_\mathrm{s} = \mathbf{m} \cdot (\mathbf{s}_\mathrm{a} - \mathbf{s}_\mathrm{b}) $)", 
           ylabel=r"$t_\mathrm{s}$ (min)",   # $\mathbf{m} \cdot \mathbf{x}_\mathrm{s}$
           title="Second phase: to selective point", xscale="log", yscale="log")
    ax.legend(**leg_kwargs)
    return fig, axes

In [ ]:
fig, axes = plot_convergence_dynamics(
    convtime_res["tser"], 
    convtime_res["mdotsers"], 
    convtime_res["mdotab_predictions"],
    convtime_res["mdotds_predictions"], 
    convtime_res["tds_predictions"]
)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder1, "supfig_ibcm1_2d_convergence_time_example.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_convergence_time_scaling(
    convtime_res["eps_axes"], 
    convtime_res["time_grids"]
)
fig.tight_layout(w_pad=0.4)
if do_save_plots:
    fig.savefig(pj(panels_folder1, "supfig_ibcm1_2d_convergence_time_scaling.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# This would be a good place to split the figure in two.
First supp fig is about the 2D toy model, second supp fig is about Gaussian vs non-Gaussian backgrounds in higher dimensions and eigenvalues for stability of fixed points. 

In [ ]:
def plot_3d_series(vs, dim_idx=[0, 1, 2], skp=10, transient=0):
    r""" # Function to plot three dimensions of vectors of synaptic weights,
    e.g. $\mathbf{m}^j$ or $\mathbf{w}^j$. Let the user label the plot.

    Args:
        vs (np.ndarray): time series of vector of each neuron,
            indexed [time, neuron, dimension].
        dims_idx (list of 3 ints): list of the three dimensions to plot.
            Defaults to the first three.
        transient (int): first time step to plot
        skp (int): plot only every skp time points
    """
    # Plot a sample of points for each neuron
    if len(dim_idx) != 3:
        raise ValueError("dim_idx should contain exactly three integers. ")
    if transient is None:
        transient = vs.shape[0] // 2
    tslice = slice(transient, None, skp)
    n_neu = vs.shape[1]
    #neurons_palette = sns.color_palette("magma", n_colors=n_neu)
    neurons_palette = sns.husl_palette(n_colors=n_neu, h=0.3, s=0.9, l=0.4, as_cmap=False)

    # 3D plot of the synaptic weight vectors of the different neurons
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(0, 0, 0, color="k", marker="o", ls="none", ms=2)
    for i in range(n_neu):
        #ax.scatter(vs[tslice, i, dim_idx[0]], vs[tslice, i, dim_idx[1]], vs[tslice, i, dim_idx[2]],
        #           s=2, alpha=0.5, color=neurons_palette[i], label="Neuron {}".format(i))
        ax.plot(vs[tslice, i, dim_idx[0]], vs[tslice, i, dim_idx[1]], vs[tslice, i, dim_idx[2]], 
                alpha=0.8, color=neurons_palette[i], label="Neuron {}".format(i), ls="-", lw=2.0)
    ax.view_init(azim=45, elev=30)
    #ax.tick_params(pad=-3)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    ax.set_xlabel(r"$\overline{m}_1$", labelpad=-16)
    ax.set_ylabel(r"$\overline{m}_2$", labelpad=-16)
    ax.set_zlabel(r"$\overline{m}_3$", labelpad=-16)
    return fig, ax


In [ ]:
def plot_hgammas_series(tser, hser, color_palette=neuron_colors_full, skp=25, 
                        clr_back=back_palette[-1], force_highlights=None):
    # Show three neurons
    fig = plt.figure()
    gs = fig.add_gridspec(3, 4)
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.25, 
                        plt.rcParams["figure.figsize"][1])
    ax = fig.add_subplot(gs[:, :3])
    axi = fig.add_subplot(gs[1, 3:])
    
    #ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
    dt_u = 10.0  # ms
    t_axis = tser*dt_u/1000/60  # minutes
    n_i_ibcm = hser.shape[1]
    n_b = hser.shape[2]
    legend_styles = [[0,]*6, [0,]*6, [0,]*6]
    step = n_i_ibcm//(2*3)
    start = 5
    if force_highlights is None:
        i_highlights = np.linspace(start, n_i_ibcm - 2*step + start, 3, dtype=int)  # Neurons to highlight
        transient = 2 * (hser.shape[0] // 3)
        specifs = np.argmax(np.mean(hser[transient:], axis=0), axis=1)
        print(specifs)
        while specifs[i_highlights][0] == specifs[i_highlights][1]:
            i_highlights[1] += 1
        while (specifs[i_highlights][1] == specifs[i_highlights][2]
                or specifs[i_highlights][0] == specifs[i_highlights][2]):
            i_highlights[2] += 1
    else:
        i_highlights = force_highlights[:n_b]
    neuron_colors3 = neuron_colors_full24[[8, 17, 23]]
    plot_skp = skp
    
    # plot all other neurons first, skip some points
    for i in range(n_i_ibcm):
        if i in i_highlights: 
            continue
        else: 
            for j in range(n_b):
                ax.plot(t_axis[::plot_skp], hser[::plot_skp, i, j], color=clr_back, 
                    ls="-", alpha=1.0-0.1*j, lw=plt.rcParams["lines.linewidth"]-j*0.1, zorder=i*n_b+j)
    
    # Now plot the highlighted neuron
    lstyles = ["-.", ":", "-"]
    for j in range(n_b):
        for i in range(len(i_highlights)):
            li, = ax.plot(t_axis[::plot_skp], hser[::plot_skp, i_highlights[i], j], 
                          color=neuron_colors3[i], ls=lstyles[j], alpha=1.0-0.15*j, 
                          lw=plt.rcParams["lines.linewidth"]*1.5-j*0.1, zorder=i*n_b+j+n_i_ibcm*n_b+3)
            legend_styles[i][j] = li
    
    # Annotate with analytical results
    #for j in range(2):
    #    ax.axhline(analytical_cs_cn[j], lw=1.0, ls="-.", color="k")
    #ax.annotate(r"Analytical $c_{ns}$", xy=(t_axis[-5], analytical_cs_cn[1]-0.4), 
    #            ha="right", va="top", color="k", size=6)
    #ax.annotate(r"Analytical $c_s$", xy=(t_axis[-5], analytical_cs_cn[0]+0.4), 
    #            ha="right", va="bottom", color="k", size=6)
    ax.set(xlabel="Time (min)", 
           ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")
    
    axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
    for i in range(len(i_highlights)):
        for j in range(n_b):
            li = legend_styles[i][j]
            axi.plot([0.0+j, 0.5+j], [0.8*i, 0.8*i], color=li.get_color(), 
                     ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
    for side in ["bottom", "left", "top", "right"]:
        axi.spines[side].set_visible(False)
    axi.tick_params(axis="both", length=0, pad=3)
    axi.set_xticks(np.arange(0.25, n_b+0.25, 1.0))
    #axi.set_xticklabels([r"$\mathbf{\bar{m}}^i \cdot \mathbf{s}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{s}_b$"])
    axi.set_xticklabels(list(range(0, n_b)))
    axi.set_xlabel(r"Odor $\gamma$", size=6, labelpad=3)
    axi.xaxis.set_label_position('top') 
    axi.set_yticks([0, 0.8, 1.6])
    axi.invert_yaxis()
    axi.set_yticklabels(["${}$".format(i) for i in i_highlights])
    #axi.set_yticklabels(["Neuron 2", "Neuron 1"])
    axi.set_ylabel("Neuron $i$", size=6, labelpad=1)

    gs.tight_layout(fig, w_pad=-0.2)

    return fig, ax, axi


In [ ]:
def plot_hgammas_sums(tser, hser, moments, color_palette=neuron_colors_full, skp=25, hs_hn=None):
    dt_u = 10.0  # ms
    t_axis = tser*dt_u/1000/60  # minutes
    hgamma_sums = np.sum(hser, axis=2)  # Should be 1 / avgnu
    hgamma2_sums = np.sum(hser**2, axis=2)  # Should be 1 / sigma2
    n_b = hser.shape[2]
    
    fig, axes = plt.subplots(2, 1, sharex=True)
    axes = axes.flatten()
    for i in range(hgamma_sums.shape[1]):
        axes[0].plot(t_axis[::skp], hgamma_sums[::skp, i], color=neuron_colors_full[i], lw=1.0, alpha=0.8)
        axes[1].plot(t_axis[::skp], hgamma2_sums[::skp, i], color=neuron_colors_full[i], lw=1.0, alpha=0.8)
    # Labels
    axes[0].set_ylabel(r"$\sum_{\gamma} \, \bar{h}_\gamma$")
    axes[1].set(ylabel=r"$\sum_{\gamma} \, \bar{h}_\gamma$", xlabel="Time (min)")
    
    # Analytical predictions for steady-state
    if hs_hn is None:  # Gaussian fixed point
        h_line = 1.0 / moments[0]
        h_label = r"$1 / \langle c \rangle$"
        h2_line = 1.0 / moments[1]
        h2_label = r"$1 / \sigma_c^2$"
        h_line_props = dict(xy=(t_axis[-1], 0.68 * h_line), ha="right", va="top")
        h2_line_props = dict(xy=(t_axis[-1], 0.85 * h2_line), ha="right", va="top")
    else:
        # h_s * 1 + h_{ns}*(N_B-1)
        h_line = hs_hn[0] + (n_b - 1) * hs_hn[1]
        h_label = r"$h_{\mathrm{sp}} + (N_\mathrm{B}-1) h_{\mathrm{ns}}$"
        h2_line = hs_hn[0]**2 + (n_b - 1) * hs_hn[1]**2
        h2_label = r"$h_{\mathrm{sp}}^2 + (N_\mathrm{B}-1) h_{\mathrm{ns}}^2$"
        h_line_props = dict(xy=(t_axis[-1], 0.6 * h_line), ha="right", va="top")
        h2_line_props = dict(xy=(t_axis[-1], 0.8 * h2_line), ha="right", va="top")
    
    axes[0].axhline(h_line, ls="--", color="k")
    axes[0].annotate(h_label, **h_line_props)
    axes[1].axhline(h2_line, ls="--", color="k")
    axes[1].annotate(h2_label, **h2_line_props)
    
    return fig, axes

# Panel 2A-B: Non-isolated fixed points with Gaussian 3D background

In [ ]:
# Import results
examples_file = np.load(pj(data_folder, "importance_thirdmoment_examples.npz"))
tser_ibcm = examples_file["tser"]
hgammaser2 = examples_file["hgammaser2"]
mbarser2 = examples_file["mbarser2"]
h_saddle2 = examples_file["h2"][0]  # float
back_vecs_example = examples_file["back_components"]
moments_conc2 = examples_file["moments2"]

In [ ]:
fig, ax = plot_3d_series(mbarser2, dim_idx=[0, 1, 2], skp=1)

scale = 3
vecs = back_vecs_example.copy()
orig = np.zeros([vecs.shape[0], 3])
for i in range(vecs.shape[0]):
    vecs[i] = vecs[i] / np.sqrt(np.sum(vecs[i]**2)) * scale
ax.quiver(*orig, *(vecs[:, :3].T), color="k", lw=1.0)

axlabels = (ax.xaxis.label, ax.yaxis.label, ax.zaxis.label)
if do_save_plots:
    fig.savefig(pj(panels_folder2, "supfig_ibcm2_gaussian_m_3d.pdf"), 
               transparent=True, dpi=200, bbox_inches="tight", bbox_extra_artists=axlabels)
plt.show()
plt.close()

In [ ]:
# dot products time series
hgammaser2 = examples_file["hgammaser2"]

fig, ax, axi = plot_hgammas_series(tser_ibcm, hgammaser2, skp=25, force_highlights=[6, 17, 29])
if do_save_plots:
    fig.savefig(os.path.join(panels_folder2, "supfig_ibcm2_gaussian_hgamma_series.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Show sum of hbars and sum of hbar_squared, to prove these quantities reach the expected fixed points
fig, axes = plot_hgammas_sums(tser_ibcm, hgammaser2, moments_conc2)
if do_save_plots:
    fig.savefig(os.path.join(panels_folder2, "supfig_ibcm2_gaussian_hgamma_sums_series.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()


# Panel 2C-D: Three fixed points with non-Gaussian 3D background

In [ ]:
# Import results
examples_file = np.load(pj(data_folder, "importance_thirdmoment_examples.npz"))
tser_ibcm = examples_file["tser"]
hgammaser3 = examples_file["hgammaser3"]
mbarser3 = examples_file["mbarser3"]
h_specif3, h_nonspecif3, h_saddle3 = examples_file["h3"]
moments_conc3 = examples_file["moments3"]

In [ ]:
fig, ax = plot_3d_series(mbarser3, dim_idx=[0, 1, 2], skp=1)

scale = 3
vecs = back_vecs_example.copy()
orig = np.zeros([vecs.shape[0], 3])
for i in range(vecs.shape[0]):
    vecs[i] = vecs[i] / np.sqrt(np.sum(vecs[i]**2)) * scale
ax.quiver(*orig, *(vecs[:, :3].T), color="k", lw=1.0)

axlabels = (ax.xaxis.label, ax.yaxis.label, ax.zaxis.label)
if do_save_plots:
    fig.savefig(pj(panels_folder2, "supfig_ibcm2_thirdmoment_m_3d.pdf"), 
               transparent=True, dpi=200, bbox_inches="tight", bbox_extra_artists=axlabels)
plt.show()
plt.close()

In [ ]:
# dot products time series
hgammaser3 = examples_file["hgammaser3"]

# Previous run showed that highlighting neurons 5, 17, 29 gives one
fig, ax, axi = plot_hgammas_series(tser_ibcm, hgammaser3, skp=25, force_highlights=[6, 17, 29])

ax.axhline(h_specif3, color="k", ls="-.", lw=1.0, zorder=32*3+1)
ax.axhline(h_nonspecif3, color="k", ls="-.", lw=1.0, zorder=32*3+1)
ax.axhline(h_saddle3, color="k", ls=":", lw=0.75, zorder=32*3+1)

# Annotate
hgammas_names = [r"Analytical $\bar{h}_\mathrm{ns}$", r"Analytical $\bar{h}_\mathrm{sp}$", "Saddle"]
analytical_hgammas = [h_nonspecif3, h_specif3, h_saddle3]
alignments = ["top", "bottom", "bottom"]
margins = [-0.2, 0.1, 0.0]
for j in range(3):
    t_coord = tser_ibcm[-1]*dt_u/1000/60
    ax.annotate(hgammas_names[j], xy=(t_coord, analytical_hgammas[j]+margins[j]), 
                ha="right", va=alignments[j], color="grey", size=6, zorder=100)
    

if do_save_plots:
    fig.savefig(os.path.join(panels_folder2, "supfig_ibcm2_thirdmoment_hgamma_series.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Show sum of hbars and sum of hbar_squared, to prove these quantities reach the expected fixed points
fig, axes = plot_hgammas_sums(tser_ibcm, hgammaser3, moments_conc3, hs_hn=[h_specif3, h_nonspecif3])
if do_save_plots:
    fig.savefig(os.path.join(panels_folder2, "supfig_ibcm2_thirdmoment_hgamma_sums_series.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()


# Figure 3: IBCM stability analysis, different backgrounds

In [ ]:
def plot_max_eigenvalues(eig_values, specif_keys):
    fig, ax = plt.subplots()
    reals, imags = np.real(eig_values), np.imag(eig_values)
    eig_nspecif = np.asarray([len(s) for s in specif_keys], dtype=int)
    highlights = (eig_nspecif == 1).astype(bool)
    n_b = eig_nspecif.max()
    high_saddle = np.logical_or((eig_nspecif == 0), (eig_nspecif == n_b)).astype(bool)
    ax.axvline(0.0, ls="--", color="k", lw=1.0)
    ax.axhline(0.0, ls="--", color="k", lw=1.0)
    scaleup = 1e3
    fixed_color = model_colors["ibcm"]
    ax.plot(reals[highlights]*scaleup, imags[highlights]*scaleup, marker="*", 
            mfc=fixed_color, mec=fixed_color, 
            ls="none", label="One odor", ms=9, alpha=0.8)
    saddle_color = "xkcd:light red"
    ax.plot(reals[high_saddle]*scaleup, imags[high_saddle]*scaleup, marker="o", 
            mfc=saddle_color, mec=saddle_color, alpha=0.6,
            ls="none", label="All odors equally", ms=6)
    if np.allclose(imags, 0.0):
        ax.set_ylim([-0.02, 0.04])
    nonhigh = np.logical_and(~highlights, ~high_saddle)
    ax.plot(reals[nonhigh]*scaleup, imags[nonhigh]*scaleup, marker="s", mfc=(0, 0, 0, 0.6), mec="k", 
           ls="none", label="2+ odors", ms=3.5)
    for side in ("top", "right"):
        ax.spines[side].set_visible(False)
    leg_kwargs = dict(title_fontsize=plt.rcParams["legend.fontsize"])
    ax.legend(title="Specificity", **leg_kwargs)
    ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{max}})$    ($\times 10^{-3}$)", 
          ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{max}})$     ($\times 10^{-3}$)")
    return fig, ax

In [ ]:
# Eigenvalues on weakly non-Gaussian 3 odors
with open(pj(data_folder, "ibcm_eigenvalues_keys_non-gaussian_example.json"), "r") as h:
    ibcm_specif_keys_nongauss = json.load(h)
ibcm_max_eig_nongauss = np.load(pj(data_folder, "sample_non-gaussian_simulation.npz"))["ibcm_eig_values"]

fig, ax = plot_max_eigenvalues(ibcm_max_eig_nongauss, ibcm_specif_keys_nongauss)
ax.set_title("Weakly non-Gaussian background")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder3, "supfig_ibcm3_eigenvalues_non-gaussian.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Eigenvalues on turbulent 6 odors
with open(pj(data_folder, "ibcm_eigenvalues_keys_turbulent_example.json"), "r") as h:
    ibcm_specif_keys_turbulent = json.load(h)
ibcm_max_eig_turbulent = np.load(pj(data_folder, "sample_turbulent_ibcm_simulation.npz"))["ibcm_eig_values"]

fig, ax = plot_max_eigenvalues(ibcm_max_eig_turbulent, ibcm_specif_keys_turbulent)
ax.set_title("Turbulent background")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder3, "supfig_ibcm3_eigenvalues_turbulent.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Eigenvalues on log-normal TODO
# Eigenvalues on turbulent 6 odors
with open(pj(data_folder, "ibcm_eigenvalues_keys_lognormal_example.json"), "r") as h:
    ibcm_specif_keys_turbulent = json.load(h)
ibcm_max_eig_turbulent = np.load(pj(data_folder, "sample_lognormal_simulation.npz"))["ibcm_eig_values"]

fig, ax = plot_max_eigenvalues(ibcm_max_eig_turbulent, ibcm_specif_keys_turbulent)
ax.set_title("Log-normal background")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder3, "supfig_ibcm3_eigenvalues_lognormal.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()